# Eden AI

Eden AI is an AI consulting company that was founded to use its resources to empower people and create impactful products that use AI to improve the quality of life for individuals, businesses and societies at large.

This example goes over how to use LangChain to interact with Eden AI models



## Calling a model


Find a feature and model  on the EdenAI website.



### text generation

In [42]:
from langchain.llms import EdenAI
from langchain import PromptTemplate, LLMChain

llm=EdenAI(feature="text" ,sub_feature="generation" , params={"providers": "openai","temperature" : 0.2,"max_tokens" : 250} )

prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""

llm(prompt)

" No, a dog cannot drive a car.\n\nReasoning: \n\n1. Driving a car requires a driver to have a valid driver's license. \n2. Dogs are not legally allowed to obtain a driver's license. \n3. Therefore, a dog cannot drive a car."

### image generation

In [43]:
import base64
from io import BytesIO
from PIL import Image
import json
def print_base64_image(base64_string):
    # Decode the base64 string into binary data
    decoded_data = base64.b64decode(base64_string)

    # Create an in-memory stream to read the binary data
    image_stream = BytesIO(decoded_data)

    # Open the image using PIL
    image = Image.open(image_stream)

    # Display the image
    image.show()

In [44]:
text2image = EdenAI(
    feature="image" ,sub_feature="generation" ,
    params={
    "providers": "openai",
    "text": "this is a test of Eden AI",
    "resolution" : "512x512"
    }
)

In [45]:
image_output

'[{"image": "...", "image_resource_url": "https://d14uq1pz7dzsdq.cloudfront.net/d15f928d-c766-44e5-8884-82fa8f7e50e5_.png?Expires=1690886567&Signature=nTjfo8DgVKrChg2q9wJyQPoBqPfUwLci2ymLiwfeZVdCDbFrLEshSE8mT--xBahwGdGpHWuoyRBxsth8m~7zwLi3AyCLIYK68t2jRmOMnxcYiYoradHP~Y5v55wDI3WJFBX-IZNZwiERuSOrEoSUSmV4HzMrpLloTGVSoVXvljqi6pPA2tkOdzna6Bh2YurIcFRfqyCLrSUeU-VGv1uLqT5jQiIx1BSp7qQj1Yj3QVSugdYdQr9yr1osQmfcZzMn86CihSckQZkXMHFld6O1mkZvPPZrimdmgWklyOLDFLMgVfR806PE9XaDKEbbGSNnDa5TTZgoDK~EbIW-K3KtRQ__&Key-Pair-Id=K1F55BTI9AHGIK"}]'

In [46]:
image_output = text2image("A cat riding a motorcycle by Picasso")

#to dict 

image=json.loads(image_output)

In [47]:
print_base64_image(image[0]["image"])

### text generation with callback

In [56]:
from langchain.llms import EdenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = EdenAI(
    callbacks=[StreamingStdOutCallbackHandler()],
    feature="text" ,sub_feature="generation" , params={"providers": "openai","temperature" : 0.2,"max_tokens" : 250}
)
prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""
print(llm(prompt))

 No, a dog cannot drive a car.

Reasoning: 

1. Driving a car requires a person to have a valid driver's license. 
2. Dogs are not able to obtain a driver's license. 
3. Therefore, a dog cannot drive a car.


## Chaining Calls

In [49]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [50]:
llm = EdenAI(
feature="text" ,sub_feature="generation" , params={"providers": "openai","temperature" : 0.2,"max_tokens" : 250}
)
text2image = EdenAI(
feature="image" ,sub_feature="generation" , params={"providers": "openai", "resolution" : "512x512"}
)

In [51]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [52]:
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a description of a logo for this company: {company_name}, the logo should not contain text at all ",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [53]:
third_prompt = PromptTemplate(
    input_variables=["company_logo_description"],
    template="{company_logo_description}",
)
chain_three = LLMChain(llm=text2image, prompt=third_prompt)

In [54]:
# Run the chain specifying only the input variable for the first chain.
overall_chain = SimpleSequentialChain(
    chains=[chain, chain_two, chain_three], verbose=True
)
catchphrase = overall_chain.run("hats")
print(catchphrase)



> Entering new SimpleSequentialChain chain...




Headwear Haven
.

The logo for Headwear Haven should be a simple, yet eye-catching design. It should feature a stylized image of a hat, such as a baseball cap, with a bright and vibrant color palette. The hat should be facing forward, with a curved brim and a slightly raised crown. The colors should be a combination of blues, greens, and yellows, with a hint of orange to give it a modern and playful feel. The overall design should be bold and inviting, conveying the message that Headwear Haven is the perfect place to find stylish and unique headwear.
[{"image": "...", "image_resource_url": "https://d14uq1pz7dzsdq.cloudfront.net/f08d3e13-33bc-4e37-8b80-54d920b352bb_.png?Expires=1690887031&Signature=kTn9m5MdtaOVK-65LmULD2~oUJRiePOI4gm3P5wg07jtl6-bxeh4ABBTxZhKAu9BMJasczaLmGcYjHsdsu~m95R7HIhbgBqoh6UYOOdycz2GGZinTBQIp6gjf30CBFIa8FLMJQgoDrgdK~lGF6BXeYE0cfq9cxx~30A5934kYuA3qWzTOIJbmNLO04~VhBwNhdmIMp8MA2XpAZR1BipA0gaMBUKV019TGqME1Exfgw1G2LMqciSAmvYcVMlaDPFEi3hTmIvIQjJNNes2ok4P1JX1pZRqiWUrDqS

In [55]:
import json
#str to dict
result= json.loads(catchphrase)

#print the image
print_base64_image(result[0]['image'])